Part 1

In [2]:
from gurobipy import *
import pandas as pd

In [10]:
# build a new model
eg1 = Model("eg1")  # build a new model, name it as "eg1"

x1 = eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x1')
x2 = eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x2")

Academic license - for non-commercial use only - expires 2021-05-30
Using license file /Users/terrylu/gurobi.lic


In [11]:
# setting the objective function
# use GRB.MAXIMIZE for a maximization problem
eg1.setObjective(700 * x1 + 900 * x2, GRB.MAXIMIZE) 

# add constraints and name them
eg1.addConstr(3 * x1 + 5 * x2 <= 3600, "resource_wood")
eg1.addConstr(x1 + 2 * x2 <= 1600, "resource_labor")
eg1.addConstr(50 * x1 + 20 * x2 <= 48000, "resource_machine")

<gurobi.Constr *Awaiting Model Update*>

In [12]:
eg1.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xa395d65c
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.02s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.04 seconds
Optimal objective  7.894736842e+05


In [13]:
for var in eg1.getVars():
    print(var.varName, '=', var.x)
print("objective value =", eg1.objVal)

x1 = 884.2105263157895
x2 = 189.47368421052633
objective value = 789473.6842105263


In [14]:
products = range(2)  # 2 products    
resources = range(3)  # 3 resources

prices = [700, 900]    
resource_consumptions = [[3 , 5 ],
                         [1 , 2 ],
                         [50, 20]]
resource_limitations = [3600, 1600, 48000]

In [15]:
eg1_decoupling = Model("eg1_decoupling")

x = []
for i in products:
    x.append(eg1_decoupling.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x' + str(i)))

eg1_decoupling.setObjective(quicksum(prices[i] * x[i] for i in products) , GRB.MAXIMIZE) 

# add constraints and name them
eg1_decoupling.addConstrs((quicksum(resource_consumptions[j][i] * x[i] for i in products) 
                           <= resource_limitations[j] for j in resources), "Resource_limitation")

eg1_decoupling.optimize()

for var in eg1_decoupling.getVars():
    print(var.varName, '=', var.x)
print("objective value =", eg1_decoupling.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xa395d65c
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.03s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.05 seconds
Optimal objective  7.894736842e+05
x0 = 884.2105263157895
x1 = 189.47368421052633
objective value = 789473.6842105263


Part 2

In [7]:
basic_info = pd.read_excel('IP_dataset.xlsx', 'Basic information')
cities = range(len(basic_info['City']))
markets = range(len(basic_info['Market']))

city_info = pd.read_excel('IP_dataset.xlsx', 'City\'s information')
operating_costs = city_info['Operating cost']
capacities = city_info['Capacity']

market_info = pd.read_excel('IP_dataset.xlsx', 'Market\'s information')
demands = market_info['Demand']

shipping_info = pd.read_excel('IP_dataset.xlsx', 'Shipping cost', index_col = 0)
shipping_costs = []
for i in shipping_info.index:
    shipping_costs.append(list(shipping_info.loc[i]))

In [11]:
shipping_costs

[[2.4, 3.25, 4.05, 5.25, 6.95],
 [3.5, 2.3, 3.25, 6.05, 5.85],
 [4.8, 3.4, 2.85, 4.3, 4.8],
 [6.8, 5.25, 4.3, 3.25, 2.1],
 [5.75, 6.0, 4.75, 2.75, 3.5]]

In [21]:
eg2 = Model("eg2")    # build a new model
    
# add variables as a list
x = []
for j in cities:
    x.append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(j+1)))
             
y = []
for i in markets:
    y.append([])
    for j in cities:
        y[i].append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "y" + str(i+1) + str(j+1)))
        

In [20]:

# setting the objective function 
eg2.setObjective(
    quicksum(operating_costs[j] * x_method1[j] for j in cities) 
    +  quicksum(quicksum(shipping_costs[i][j] * y[i][j] for j in cities) for i in markets)
    , GRB.MINIMIZE) 

# add constraints and name them
eg2.addConstrs((quicksum(y[i][j] for i in markets) <= capacities[j] * x[j] 
                for j in cities), "productCapacity")

eg2.addConstrs((quicksum(y[i][j] for j in cities) >= demands[i]
                for i in markets), "demand_fulfillment")
    
eg2.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0x263201cb
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+00, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 25 continuous, 5 integer (5 binary)

Root relaxation: objective 2.528000e+05, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 252800.000    0    5          - 252800.000      -     -    0s
H    0     0                    313600.00000 252800.000  19.4%     -    0s
H    0     0                    280400.00000 252800.000  9.84%     -    0s
     

In [15]:
print("Result:")

for j in cities:
    print(x[j].varName, '=', x[j].x)
# head of the result table
print("\tMarket1\tMarket2\tMarket3\tMarket4\tMarket5")

for j in cities:
    # mark which product is printed now
    print("City" + str(j+1), "\t", end="")
    for i in markets:
        # print values of each kind of product
        if len(str(y[i][j].x)) < 7:
            print(y[i][j].x, "\t", end="")
        else:
            print(y[i][j].x, "", end="")
    print("")    # use for change line

print("z* =", eg2.objVal)    # print objective value

Result:
x1 = 0.0
x2 = 1.0
x3 = 0.0
x4 = 1.0
x5 = 1.0
	Market1	Market2	Market3	Market4	Market5
City1 	0.0 	0.0 	0.0 	0.0 	0.0 	
City2 	7999.999999999998 12000.000000000002 0.0 	0.0 	0.0 	
City3 	0.0 	0.0 	0.0 	0.0 	0.0 	
City4 	0.0 	0.0 	8000.0 	0.0 	17000.0 
City5 	0.0 	0.0 	1000.0 	14000.0 0.0 	
z* = 268950.0


In [22]:
y

[[<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>],
 [<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>],
 [<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>],
 [<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>],
 [<gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update*>,
  <gurobi.Var *Awaiting Model Update